In [1]:
import copy

input_file = "data/input.txt"

def _parse_work_item(line):
    line = line.strip("{}")
    params = line.split(",")
    params = [param.split("=") for param in params]
    item = {param[0]: int(param[1]) for param in params}
    return item

def _parse_workflow(line):
    name, rules = line.split("{")
    rules = rules.strip("}")
    rules = rules.split(",")
    rules = [rule.split(":") for rule in rules[:-1]] + [rules[-1]]
    parsed_rules = []
    for (rule, destination) in rules[:-1]:
        if ">" in rule:
            k, v = rule.split(">")
            gt = True
        else:
            k, v = rule.split("<")
            gt = False
        parsed_rules.append((k, gt, int(v), destination))
    parsed_rules.append(rules[-1])
    return name, parsed_rules

def parse_workflows(lines):
    workflows = [w.strip() for w in lines.split("\n")]
    workflows = [_parse_workflow(w) for w in workflows]
    workflows = {name: rules for name, rules in workflows}
    return workflows

def parse_work_items(lines):
    work_items = [w.strip() for w in lines.split("\n")]
    work_items = [_parse_work_item(w) for w in work_items]
    return work_items

def apply_workflow(item, workflow):
    rules = workflow[:-1]
    next_workflow = workflow[-1]
    for rule in rules:
        param, gt, value, destination = rule
        if gt and item[param] > value or not gt and item[param] < value:
            return destination
    return next_workflow

def get_range_size(range):
    range_size = 1
    for k in range:
        start, end = range[k]
        range_size *= ((end+1) - start)
    return range_size

def find_all_ranges(ranges, status):
    if status == "A":
        return get_range_size(ranges)
    elif status == "R":
        return 0
    size = 0
    for param, gt, limit, destination in workflows[status][:-1]:
        new_ranges = copy.deepcopy(ranges)
        if gt and new_ranges[param][1] > limit:
            new_ranges[param][0] = max(new_ranges[param][0], limit+1)
            size += find_all_ranges(new_ranges, destination)
            ranges[param][1] = min(ranges[param][1], limit)
        elif not gt and new_ranges[param][0] < limit:
            new_ranges[param][1] = min(new_ranges[param][1], limit-1)
            size += find_all_ranges(new_ranges, destination)
            ranges[param][0] = max(ranges[param][0], limit)
    final_dest = workflows[status][-1]
    size += find_all_ranges(ranges, final_dest)
    return size

with open(input_file, 'r') as f:
    [workflows, work_items] = f.read().split("\n\n")
    workflows = parse_workflows(workflows)
    work_items = parse_work_items(work_items)

    ans1 = 0
    for item in work_items:
        status = 'in'
        while status not in ["A", "R"]:
            status = apply_workflow(item, workflows[status])
        if status == "A":
            ans1 += sum(item.values())

    initial_ranges = {
        k: [1, 4000] for k in "xmas"
    }

    ans2 = find_all_ranges(initial_ranges, 'in')

    print(f"{ans1 = }")
    print(f"{ans2 = }")

ans1 = 367602
ans2 = 125317461667458
